In [ ]:
from my_functions import read_obsfcstana
import numpy as np
from mapper_functions import plot_global_tight
import matplotlib.pyplot as plt


In [ ]:
expt2 = 'DAv7_M36_ASCAT_type_13_comb_fp'

path = f'/discover/nobackup/amfox/Experiments/{expt2}/{expt2}/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M04'
# path = f'/discover/nobackup/amfox/Experiments/{expt}/{expt}/scratch'
path = '../test_data'

file_name_start = f'{expt2}.ens_avg.ldas_ObsFcstAna.20150402_00'
printflag = False

date_time2, obs_species2, obs_tilenum2, obs_lon2, obs_lat2, obs_obs2, obs_obsvar2, obs_fcst2, obs_fcstvar2, obs_ana2, obs_anavar2 = read_obsfcstana(path, file_name_start, printflag)

numobs2 = len(obs_obs2[obs_species2 > 4])
obarray2 = np.empty([numobs2, 3])
obarray2[:, 1] = obs_lon2[obs_species2 > 4]
obarray2[:, 2] = obs_lat2[obs_species2 > 4]
obarray2[:, 0] = obs_obs2[obs_species2 > 4]

plot_global_tight(obarray2,False,f'Observations: {expt2}','(SFDS)')

# Find the closest obs_lon2 value to 30.8091
# Find the closest obs_lat2 value to -0.6283

test = np.abs(obs_lon2 - 30.8091) + np.abs(obs_lat2 + 0.6283)
print(np.min(test))
print(np.where(test == np.min(test)))
print(obs_obs2[np.where(test == np.min(test))])

In [ ]:
obarray2[:, 0] = obs_obs2[obs_species2 > 4] - obs_fcst2[obs_species2 > 4]
plot_global_tight(obarray2,False,f'OmF: {expt2}','(SFDS)')

In [ ]:
obarray2[:, 0] = obs_ana2[obs_species2 > 4] - obs_fcst2[obs_species2 > 4]
plot_global_tight(obarray2,False,f'Increment: {expt2}','(SFDS)', -0.1, 0.1)


In [ ]:
data_2015 = np.load('../test_data/AMS_24/increment_time_series_analyzer/smap_ascat_multivariate_test_obsfcstana_extend_datetime_2015.npz', allow_pickle=True)

date_time3 = data_2015['date_time']
obs_species3 = data_2015['obs_species']
obs_tilenum3 = data_2015['obs_tilenum']
obs_lon3 = data_2015['obs_lon']
obs_lat3 = data_2015['obs_lat']
obs_obs3 = data_2015['obs_obs']
obs_fcst3 = data_2015['obs_fcst']
obs_ana3 = data_2015['obs_ana']

In [ ]:
from datetime import datetime

# Calculate obs minus fcst
obs_minus_fcst3 = []
obs_minus_ana3 = []
obs_inc3 = []
new_date_time3= []
for i in range(len(obs_obs3)):
    obs_minus_fcst_chunk = obs_obs3[i] - obs_fcst3[i]
    obs_minus_fcst3.append(obs_minus_fcst_chunk)
    obs_minus_ana_chunk = obs_obs3[i] - obs_ana3[i]
    obs_minus_ana3.append(obs_minus_ana_chunk)
    obs_inc_chunk = obs_ana3[i] - obs_fcst3[i]
    obs_inc3.append(obs_inc_chunk)
    
    new_date_time3.append(datetime(
    date_time3[i]['year'][0],
    date_time3[i]['month'][0],
    date_time3[i]['day'][0],
    date_time3[i]['hour'][0],
    date_time3[i]['min'][0],
    date_time3[i]['sec'][0]
    ))

In [ ]:
print(obs_minus_fcst3[0])

In [ ]:

obs_minus_fcst3 = np.array(obs_minus_fcst3)
obs_minus_ana3 = np.array(obs_minus_ana3)
obs_inc3 = np.array(obs_inc3)

# Find the values for the first time stamp (2015-04-02 00:00:00) and obs_species < 5
date_time3 = np.array(new_date_time3)
date_time_2015_04_02 = datetime(2015, 4, 2, 0, 0, 0)
indices = np.where((date_time3 == date_time_2015_04_02) & (obs_species3 > 4))[0]



In [ ]:

obs_minus_fcst_2015_04_02 = obs_minus_fcst3[indices]
obs_minus_ana_2015_04_02 = obs_minus_ana3[indices]
obs_inc_2015_04_02 = obs_inc3[indices]
obs_obs_2015_04_02 = obs_obs3[indices]

obarray3 = np.empty([len(obs_minus_fcst_2015_04_02), 3])
obarray3[:, 1] = obs_lon3[indices]
obarray3[:, 2] = obs_lat3[indices]

obarray3[:, 0] = obs_minus_fcst_2015_04_02
plot_global_tight(obarray3,False,f'OmF: AMS_24','(SFDS)')

obarray3[:, 0] = obs_inc_2015_04_02
plot_global_tight(obarray3,False,f'Increment: AMS_24','(SFDS)', -0.1, 0.1)

plot_global_tight(obarray2,False,f'Increment: {expt2}','(SFDS)', -0.1, 0.1)

In [ ]:
print('Length Obarray3 = ', len(obarray3))
print('Length Obarray2 = ', len(obarray2))

In [ ]:
matching_indices3 = []
matching_indices4 = []
for i in range(len(obarray2)):
    # Print i if i is a multiple of 1000
    if i % 1000 == 0:
        print(i)
    for j in range(len(obarray3)):
            if (obarray2[i][1] == obarray3[j][1] and obarray2[i][2] == obarray3[j][2]):
                matching_indices3.append(i)
                matching_indices4.append(j)
                break

print('Length matching indices = ', len(matching_indices3))

In [ ]:
test_set = obs_ana3[indices][obs_species3[indices] > 4]

x_values = obs_ana2[obs_species2 > 4][matching_indices3]
y_values = test_set[matching_indices4]

# make the points very small
plt.scatter(x_values, y_values, s=0.1)
plt.plot(np.linspace(0, 0.6, 100), np.linspace(0, 0.6, 100), color='red')
plt.xlabel(f'Ana {expt2}')
plt.ylabel(f'Ana AMS24')
title = (f'Ana {expt2} vs AMS24')
plt.title(title)
plt.savefig(f'{title}.png', dpi=300)
plt.show()

In [ ]:
test_set = obs_fcst3[indices][obs_species3[indices] > 4]

x_values2 = obs_fcst2[obs_species2 > 4][matching_indices3]
y_values2 = test_set[matching_indices4]

# make the points very small
plt.scatter(x_values2, y_values2, s=0.1)
plt.plot(np.linspace(0, 0.6, 100), np.linspace(0, 0.6, 100), color='red')
plt.xlabel(f'Fcst {expt2}')
plt.ylabel(f'Fcst AMS24')
title = (f'Fcst {expt2} vs AMS24')
plt.title(title)
plt.savefig(f'{title}.png', dpi=300)
plt.show()

In [ ]:
# make the points very small
x_values3 = x_values2 - x_values
y_values3 = y_values2 - y_values

plt.scatter(x_values3, y_values3, s=0.1)
plt.plot(np.linspace(-0.1, 0.1, 100), np.linspace(-0.1, 0.1, 100), color='red')
plt.xlabel(f'Inc {expt2}')
plt.ylabel(f'Inc AMS24')
title = (f'Inc {expt2} vs AMS24')
plt.title(title)

# Limit axes 
plt.xlim(-0.1, 0.1)
plt.ylim(-0.1, 0.1)

plt.savefig(f'{title}.png', dpi=300)
plt.show()

obarray5 = np.empty([len(x_values), 3])
obarray5[:, 1] = obs_lon2[obs_species2 > 4][matching_indices3]
obarray5[:, 2] = obs_lat2[obs_species2 > 4][matching_indices3]
obarray5[:, 0] = x_values3 - y_values3

plot_global_tight(obarray5,True,f'Inc difference: {expt2} - AMS_24','(SFDS)', -1, 1)

plt.hist(x_values3 - y_values3, bins=100)
plt.xlim(-0.1, 0.1)

plt.xlabel(f'Inc diff {expt2} - AMS24')
plt.ylabel(f'Count')
title = (f'Inc diff {expt2} - AMS24 distribution')
plt.title(title)
plt.savefig(f'{title}.png', dpi=300)
plt.show()



In [ ]:
import xarray as xr

type_10_file = '../test_data/DAv7_M36_SMAP_type_10_fp_test.catch_progn_incr.20150402_0000z.nc4'
type_13_file = '../test_data/DAv7_M36_SMAP_type_13_fp_test.catch_progn_incr.20150402_0000z.nc4'

data = xr.open_dataset(type_10_file)
data2 = xr.open_dataset(type_13_file)

# For each variable in the dataset, print the variable name, its min and max and mean values
for var in data.variables:
    print(var)
    print('Min ', data[var].min().values, data2[var].min().values)
    print('Max ', data[var].max().values, data2[var].max().values)
    print('Mean ', data[var].mean().values, data2[var].mean().values)
    print('\n')


In [ ]:

# Extract the variable called "SRFEXC_INCR"
srfexc_incr = data['SRFEXC_INCR']

# Print the first 100 non-zero values
non_zero_values = srfexc_incr.values[srfexc_incr.values != 0]
print(non_zero_values[:10])


data2 = xr.open_dataset(type_13_file)
# Extract the variable called "SRFEXC_INCR"
srfexc_incr2 = data2['SRFEXC_INCR']

# Print the first 100 non-zero values
non_zero_values2 = srfexc_incr2.values[srfexc_incr2.values != 0]
print(non_zero_values2[:10])


In [ ]:

print(np.min(non_zero_values))
print(np.max(non_zero_values))
print(np.min(non_zero_values2))
print(np.max(non_zero_values2))

